In [4]:
pwd

'/home/wsuser/work'

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GFFkMBWO15paKH1O1jX4GD_J32x0XfF7z9K6c_AH0OWX',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-kexaouraezeh26'
object_key = 'Training-20221112T024940Z-001.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [1]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

NameError: name 'streaming_body_1' is not defined

In [7]:
ls

nutrition-classification-model.tgz  Training/


In [8]:
pwd

'/home/wsuser/work'

In [9]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of Layers
from tensorflow.keras import layers #A Layer consists of a tensor-in tensor-out computation function
#Dense Layer is the regular deeply connected neural network Layer
from tensorflow.keras.layers import Dense, Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

2022-11-18 09:30:35.148460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [11]:
x_train = train_datagen.flow_from_directory(
    r'/home/wsuser/work/Training/Dataset/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    r'/home/wsuser/work/Training/Dataset/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 4138 images belonging to 5 classes.
Found 4138 images belonging to 5 classes.


In [12]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [13]:
print(x_test.class_indices) #checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [14]:
from collections import Counter as c 
c(x_train .labels)

Counter({0: 995, 1: 1374, 2: 1019, 3: 275, 4: 475})

In [15]:
model=Sequential()

2022-11-18 09:30:43.542112: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-18 09:30:43.542203: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [16]:
# Initializing the CNN classifier = Sequential()
classifier = Sequential()
# First convolution layer and pooling
classifier.add(Conv2D(32,(3, 3), input_shape=(64, 64, 3),activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the Layers
classifier.add(Flatten())
# Adding fully connected Layer a
classifier.add(Dense (units=128, activation='relu'))
classifier.add(Dense (units=5, activation='softmax')) # softmax for more than 2

In [17]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)              

In [18]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
##Fitting the model
classifier.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train), 
    epochs=20,validation_data=x_test,validation_steps = len(x_test)) # No of images in test set


Epoch 1/20


/tmp/wsuser/ipykernel_1592/2706448856.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


828/828 [==============================] - 59s 70ms/step - loss: 0.6154 - accuracy: 0.7637 - val_loss: 0.5561 - val_accuracy: 0.8057
Epoch 2/20
828/828 [==============================] - 58s 70ms/step - loss: 0.4144 - accuracy: 0.8538 - val_loss: 0.3022 - val_accuracy: 0.8792
Epoch 3/20
828/828 [==============================] - 57s 69ms/step - loss: 0.3773 - accuracy: 0.8610 - val_loss: 0.4107 - val_accuracy: 0.8473
Epoch 4/20
828/828 [==============================] - 57s 69ms/step - loss: 0.3587 - accuracy: 0.8673 - val_loss: 0.3196 - val_accuracy: 0.8736
Epoch 5/20
828/828 [==============================] - 58s 69ms/step - loss: 0.3257 - accuracy: 0.8758 - val_loss: 0.2663 - val_accuracy: 0.9072
Epoch 6/20
828/828 [==============================] - 58s 70ms/step - loss: 0.3078 - accuracy: 0.8847 - val_loss: 0.2331 - val_accuracy: 0.9203
Epoch 7/20
828/828 [==============================] - 58s 70ms/step - loss: 0.2899 - accuracy: 0.8917 - val_loss: 0.3402 - val_accuracy: 0.8731
Epo

In [20]:
ls

nutrition-classification-model.tgz  Training/


In [21]:
classifier.save('nutrition.h5')

In [22]:
ls

nutrition-classification-model.tgz  nutrition.h5  Training/


In [23]:
pwd

'/home/wsuser/work'

In [24]:
### Predicting our results
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition.h5") #Loading the model for testing
from tensorflow.keras.preprocessing import image

In [26]:
img =image.load_img(r"/home/wsuser/work/Training/Dataset/TRAIN_SET/ORANGE/100_100.jpg",grayscale=False, target_size= (64,64))
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis=0) #changing the shape =
y =np.argmax(model.predict(x),axis=1)
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
index[y[0]]

FileNotFoundError: [Errno 2] No such file or directory: '/home/wsuser/work/Training/Dataset/TRAIN_SET/ORANGE/100_100.jpg'

# IBM Deployment

In [ ]:
!pip install watson-machine-learning-client

In [ ]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"8jr_3j3cMpvuPq0YklClRwbLiNipMBpeDj98fB_xkqHi"
}

client=APIClient(wml_credentials)

In [ ]:
client

In [ ]:
def guid_space_name(client,nutrition_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==nutrition_deploy)['metadata']['id'])

# Deployment

In [ ]:
space_uid=guid_space_name(client,'nutrition_deploy')
print("Space UID " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list(200)

In [ ]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [ ]:
software_space_uid

In [ ]:
!tar -zcvf nutrition-classification-model.tgz nutrition.h5

In [ ]:
model_details=client.repository.store_model(model ='nutrition-classification-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN Model Building",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_uid
})

In [ ]:
model_id = client.repository.get_model_id(model_details)

In [ ]:
model_id

In [ ]:
ls

In [ ]:
client.repository.download(model_id,'nutrition.tar.gb')

In [ ]:
pwd